<a href="https://colab.research.google.com/github/HaroldSthid/blueprint_NLP_and_babysteps_LLMs/blob/main/NLP_LLMs_babysteps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# -*- coding: utf-8 -*-
## Análisis de Sentimiento de Películas de IMDb.ipynb

Automatically generated by Colab.

Original file is located at
    https://github.com/HaroldSthid/blueprint_NLP_and_babysteps_LLMs/blob/main/README.md

# 🎬 Análisis de Sentimiento de Películas de IMDb

¡Bienvenido! Este proyecto te ayudará a analizar reseñas de películas y descubrir patrones interesantes sobre qué hace que una película sea bien recibida.

## 📋 Paso 0: Configuración inicial

Primero, necesitamos preparar nuestro entorno instalando todas las herramientas necesarias.

In [ ]:
# Instalar todas las librerías necesarias
%pip install tensorflow tensorflow-datasets pandas numpy matplotlib seaborn wordcloud nltk scikit-learn gensim

In [ ]:
# Importar todas las librerías que vamos a utilizar
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import os

print("✅ Todas las librerías se instalaron e importaron correctamente")

## 📥 Paso 1: Cargar los datos

Vamos a cargar dos conjuntos de datos:
1. Reseñas de películas de IMDb (con etiquetas de sentimiento)
2. Metadatos de películas (información adicional como género, director, etc.)


In [ ]:
# Descargar y cargar las reseñas de IMDb
print("📥 Descargando reseñas de películas de IMDb...")
dataset, info = tfds.load('imdb_reviews', split='train', with_info=True, as_supervised=True)
print("✅ Reseñas cargadas correctamente")

# Ver información básica del dataset
print(f"ℹ️  El dataset contiene {info.splits['train'].num_examples} reseñas")

# Mostrar algunas reseñas de ejemplo
print("\n🔍 Algunas reseñas de ejemplo:")
for i, (text, label) in enumerate(dataset.take(3)):
    print(f"Reseña {i+1}: {text.numpy()[:100]}...")  # Mostrar solo los primeros 100 caracteres
    print(f"Sentimiento: {'Positivo' if label.numpy() else 'Negativo'}")
    print("---")

## 🧹 Paso 2: Limpiar y preparar el texto

Las reseñas de películas a menudo contienen HTML, signos de puntuación y otras cosas que no necesitamos para nuestro análisis. Vamos a limpiarlas.

In [ ]:
# Descargar recursos necesarios para procesamiento de texto
print("📥 Descargando recursos de procesamiento de lenguaje...")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Configurar herramientas de limpieza de texto
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    """
    Función para limpiar texto:
    1. Convierte a minúsculas
    2. Elimina etiquetas HTML
    3. Elimina signos de puntuación
    4. Elimina espacios extra
    """
    # Si es un tensor de TensorFlow, convertirlo a texto
    if isinstance(text, tf.Tensor):
        text = text.numpy().decode('utf-8')

    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'<br />', ' ', text)  # Eliminar etiquetas HTML
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)  # Eliminar puntuación
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios extras
    text = text.strip()  # Eliminar espacios al inicio y final

    return text

# Probar la función de limpieza con una reseña de ejemplo
print("🧪 Probando la limpieza de texto...")
sample_text = "This is a <br /> GREAT movie!!! It's amazing."
cleaned_text = clean_text(sample_text)
print(f"Texto original: {sample_text}")
print(f"Texto limpio: {cleaned_text}")

## 🔄 Paso 3: Procesamiento avanzado de texto

Ahora vamos a mejorar aún más nuestro texto usando técnicas avanzadas como la lematización (convertir palabras a su forma base).

In [ ]:
def lemmatize_text(text):
    """
    Función para lematizar texto (convertir palabras a su forma base)
    Ejemplo: "running" -> "run", "better" -> "good"
    """
    # Descargar recurso punkt_tab if not already downloaded
    try:
        nltk.data.find('tokenizers/punkt_tab')
    except nltk.downloader.DownloadError:
        nltk.download('punkt_tab')

    # Tokenizar (dividir en palabras)
    words = word_tokenize(text)

    # Lematizar cada palabra y eliminar palabras vacías
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    # Unir las palabras de nuevo en un texto
    return " ".join(lemmatized_words)

# Probar la lematización con un ejemplo
print("🧪 Probando lematización de texto...")
sample_text = "I was running quickly to find better solutions"
lemmatized_text = lemmatize_text(sample_text)
print(f"Texto original: {sample_text}")
print(f"Texto lematizado: {lemmatized_text}")

# Aplicar limpieza y lematización a todo el dataset
print("🔄 Procesando todas las reseñas (esto puede tomar unos minutos)...")

def process_dataset(dataset, sample_size=1000):
    """
    Procesa un conjunto de datos de reseñas
    """
    texts = []
    labels = []

    for text, label in dataset.take(sample_size):  # Usar solo una muestra para no sobrecargar
        cleaned = clean_text(text)
        lemmatized = lemmatize_text(cleaned)
        texts.append(lemmatized)
        labels.append(label.numpy())

    return texts, labels

# Procesar las reseñas
texts, labels = process_dataset(dataset, sample_size=1000)
print(f"✅ Procesamiento completado. {len(texts)} reseñas procesadas.")

## 📊 Paso 4: Análisis y Visualización de Datos

Ahora que hemos limpiado y procesado el texto, podemos empezar a explorar los datos para encontrar patrones interesantes.

### Distribución de Sentimientos

In [ ]:
# Contar la distribución de sentimientos
sentiment_counts = pd.Series(labels).value_counts()

plt.figure(figsize=(6, 4))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values)
plt.xticks([0, 1], ['Negativo', 'Positivo'])
plt.title('Distribución de Sentimientos en las Reseñas')
plt.xlabel('Sentimiento')
plt.ylabel('Número de Reseñas')
plt.show()

### Nube de Palabras para Reseñas Positivas y Negativas

In [ ]:
# Separar reseñas positivas y negativas
positive_texts = [texts[i] for i, label in enumerate(labels) if label == 1]
negative_texts = [texts[i] for i, label in enumerate(labels) if label == 0]

# Generar nube de palabras para reseñas positivas
positive_wordcloud = WordCloud(width=800, height=400, background_color='white').generate(" ".join(positive_texts))

# Generar nube de palabras para reseñas negativas
negative_wordcloud = WordCloud(width=800, height=400, background_color='white').generate(" ".join(negative_texts))

# Mostrar las nubes de palabras
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(positive_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Palabras Comunes en Reseñas Positivas')

plt.subplot(1, 2, 2)
plt.imshow(negative_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Palabras Comunes en Reseñas Negativas')

plt.show()

## 🤖 Paso 5: Entrenar un modelo de inteligencia artificial

Ahora vamos a entrenar un modelo que pueda predecir si una reseña es positiva o negativa basándose en su texto.

In [ ]:
print("🤖 Entrenando modelo de análisis de sentimiento...")

# Convertir texto a características numéricas usando TF-IDF
print("📊 Convirtiendo texto a características numéricas...")
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(texts)
y = np.array(labels)

print(f"✅ Texto convertido. Dimensiones: {X.shape}")

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"📝 Datos de entrenamiento: {X_train.shape[0]} ejemplos")
print(f"📝 Datos de prueba: {X_test.shape[0]} ejemplos")

# Entrenar modelo de regresión logística
print("🎓 Entrenando modelo...")
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

print("✅ Modelo entrenado correctamente")

# Evaluar el modelo
print("📈 Evaluando modelo...")
y_pred = model.predict(X_test)

print("🔍 Reporte de clasificación:")
print(classification_report(y_test, y_pred, target_names=['Negativo', 'Positivo']))

# Matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negativo', 'Positivo'],
            yticklabels=['Negativo', 'Positivo'])
plt.title('Matriz de Confusión')
plt.ylabel('Verdaderos')
plt.xlabel('Predicciones')
plt.show()

## 💾 Paso 6: Guardar el modelo entrenado

Vamos a guardar nuestro modelo para usarlo más tarde sin necesidad de volver a entrenarlo.

In [ ]:
print("💾 Guardando modelo y vectorizador...")

# Guardar modelo
with open('sentiment_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Guardar vectorizador
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

print("✅ Modelo y vectorizador guardados correctamente")

## 🎯 Paso 7: Probar el modelo con nuevas reseñas

Ahora vamos a probar nuestro modelo con algunas reseñas nuevas para ver cómo funciona.

In [ ]:
# Función para predecir sentimiento de nuevo texto
def predict_sentiment(text, model, vectorizer):
    """
    Predice si un texto tiene sentimiento positivo o negativo
    """
    # Limpiar y procesar el texto
    cleaned_text = clean_text(text)
    lemmatized_text = lemmatize_text(cleaned_text)

    # Convertir a características
    features = vectorizer.transform([lemmatized_text])

    # Predecir
    prediction = model.predict(features)
    probability = model.predict_proba(features)

    # Obtener confianza de la predicción
    confidence = probability[0][prediction[0]]

    return "Positivo" if prediction[0] == 1 else "Negativo", confidence

# Probar con algunas reseñas de ejemplo
test_reviews = [
    "This movie is absolutely fantastic! The acting was superb and the plot was engaging.",
    "I hated this film. It was boring and the characters were poorly developed.",
    "The movie was okay. Not great but not terrible either.",
    "This is the worst movie I have ever seen in my life. A complete waste of time.",
    "Brilliant cinematography and outstanding performances by the entire cast."
]

print("🧪 Probando el modelo con nuevas reseñas...")
for i, review in enumerate(test_reviews):
    sentiment, confidence = predict_sentiment(review, model, vectorizer)
    print(f"Reseña {i+1}: {sentiment} (confianza: {confidence:.2f})")
    print(f"Texto: {review[:80]}...")  # Mostrar solo el inicio de la reseña
    print("---")

## 📈 Paso 8: Análisis avanzado y visualización

Vamos a hacer un análisis más profundo para entender qué palabras son más importantes para determinar el sentimiento.

In [ ]:
# Obtener las palabras más importantes para cada clase
print("🔍 Analizando palabras más importantes...")

feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

# Crear un DataFrame para las palabras y sus coeficientes
word_importance = pd.DataFrame({
    'word': feature_names,
    'coefficient': coefficients
})

# Ordenar por importancia
word_importance = word_importance.sort_values('coefficient', ascending=False)

# Mostrar las 10 palabras más positivas y negativas
print("📊 Top 10 palabras para sentimiento POSITIVO:")
print(word_importance.head(10)[['word', 'coefficient']])

print("\n📊 Top 10 palabras para sentimiento NEGATIVO:")
print(word_importance.tail(10)[['word', 'coefficient']])

# Visualizar las palabras más importantes
plt.figure(figsize=(12, 8))

# Palabras positivas
top_positive = word_importance.head(10)
plt.subplot(1, 2, 1)
plt.barh(top_positive['word'], top_positive['coefficient'], color='green')
plt.title('Palabras más asociadas con reseñas POSITIVAS')
plt.xlabel('Coeficiente (mayor = más positivo)')

# Palabras negativas
top_negative = word_importance.tail(10)
plt.subplot(1, 2, 2)
plt.barh(top_negative['word'], top_negative['coefficient'], color='red')
plt.title('Palabras más asociadas con reseñas NEGATIVAS')
plt.xlabel('Coeficiente (menor = más negativo)')

plt.tight_layout()
plt.show()


## 🎉 ¡Felicidades!

Has completado exitosamente el análisis de sentimiento de películas de IMDb.

### 📋 Resumen de lo que lograste:

1. ✅ Cargaste y exploraste un dataset de reseñas de películas
2. ✅ Limpiaste y preprocesaste el texto de las reseñas
3. ✅ Visualizaste las palabras más comunes en reseñas positivas y negativas
4. ✅ Entrenaste un modelo de machine learning para predecir sentimiento
5. ✅ Evaluaste el rendimiento de tu modelo
6. ✅ Probaste el modelo con nuevas reseñas
7. ✅ Analizaste qué palabras son más importantes para determinar sentimiento

### 🚀 Próximos pasos que puedes intentar:

1. **Mejorar el modelo**: Prueba con diferentes algoritmos de machine learning
2. **Más datos**: Usa el dataset completo en lugar de solo una muestra
3. **Aplicación web**: Crea una interfaz web donde las personas puedan escribir reseñas y obtener predicciones
4. **Análisis de género**: Combina con datos de géneros de películas para ver qué géneros tienden a tener mejores reseñas

### 📚 Recursos para aprender más:

- [Curso de Machine Learning para principiantes](https://www.coursera.org/learn/machine-learning)
- [Introducción a Procesamiento de Lenguaje Natural](https://www.youtube.com/watch?v=8S3qHHUKqYk)
- [TensorFlow para principiantes](https://www.tensorflow.org/resources/learn-ml)

¡Gracias por seguir este tutorial! Si tienes preguntas o comentarios, no dudes en compartirlos.

In [ ]:
# Mensaje final
print("\n🎉 ¡Análisis completado exitosamente!")
print("📁 Los archivos del modelo se han guardado en tu entorno de Google Colab")
print("🚀 Puedes ejecutar esta celda nuevamente para reentrenar el modelo o probar con nuevas reseñas")